In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from typing import List, Optional, AsyncGenerator
from datetime import datetime
from pydantic import Field
from core import BaseCall, Msg
import instructor
from openai import AsyncOpenAI
import pandas as pd
from typing import Dict
from sampling import WeightedSampler

In [3]:
from query_planner import QueryPlan
import enum
from typing import List
from pydantic import Field
from core import user_msg
def create_enum_class(name: str, values: List[str]) -> enum.Enum:
    return enum.Enum(name, {value.upper(): value for value in values})


In [4]:
def create_search_query_class(speakers: Optional[List[str]] = None, topic_types: Optional[List[str]] = None):
    class DynamicSearchQuery(BaseCall):
        query: Optional[List[str]] = Field(description="A specific, focused search query to find semantically relevant content.")
        include_speakers: Optional[List[create_enum_class('IncludeSpeakers', speakers)]] = Field(description="Speakers to include in search results")
        exclude_speakers: Optional[List[create_enum_class('ExcludeSpeakers', speakers)]] = Field(description="Speakers to exclude from search results")
        topic_type:       Optional[List[create_enum_class('TopicType', topic_types)]]    = Field(description="Types of topics to filter by")
    return DynamicSearchQuery

In [5]:

class SearchQuery(BaseCall):
    query: Optional[List[str]] = Field(description="A specific, focused search query to find semantically relevant content.")
    include_speakers: Optional[List[str]] = Field(description="Speakers to include in search results May reference speaker groups")
    exclude_speakers: Optional[List[str]] = Field(description="Speakers to exclude from search results. May reference speaker groups")
    topic_type: Optional[List[str]] = Field(description="Types of topics to filter by")
    topic_name: Optional[List[str]] = Field(description="Topics name to include")

In [6]:
from sampling import WeightedSampler

In [7]:
from sampling import fetch_joined_data,WeightedSampler

df = await fetch_joined_data()

In [8]:
sampler = WeightedSampler(df.drop(columns=['other_speakers']),date_column='meeting_timestamp')

Building vector index...


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
samped_df = sampler.sample(n_samples=100)
topics = df['topic_name'].unique().tolist()
topic_types = df['topic_type'].unique().tolist()
speakers = df['speaker_name'].unique().tolist()

#SearchQuery = create_search_query_class(speakers, topic_types)

In [10]:
from core import system_msg,user_msg,assistant_msg

In [11]:
speakers

['Dmitriy Grankin',
 'Sergey Ryabenko',
 'Ilia Semukhin',
 'Olga Nemirovskaya',
 'Alex Shevliakov',
 'Nick Frolov',
 'nan (0)',
 'nan (1)',
 'nan',
 'Jenish Mursidinov',
 'Dmitry Grankin',
 'Vasilii Glebov',
 'Igor Vinidiktov',
 'nan (22)',
 'Сергей',
 'Nikolay',
 'Andy Black',
 'СЖ',
 'DG',
 'Сергей Жилко',
 'Lara Vargas',
 'Murat Sultanov',
 'Rick Tousseyn',
 'Umar Lateef',
 'Tatiana Sukhova',
 'Olga Mykhoparkina',
 'Matt Lewis',
 'David Sterry',
 'Nadya Gorodetskaya',
 'Igor Bessonov',
 'Ethan Noah',
 'AI Marketing Directory',
 'nan (3)',
 'nan (2)',
 'nan (19)',
 'Ilya Semukhin',
 'ISSE',
 'Özay Demirezen',
 'Ahmed Abdelaziz',
 'Sergio Goriachev',
 'Karine Stepanyan',
 'Matthew Berman',
 'nan (5)',
 'nan (6)',
 'nan (10)',
 'nan (31)',
 'nan (43)',
 'nan (38)',
 'nan (225)',
 'nan (11)',
 'nan (101)',
 'nan (205)',
 'nan (210)',
 'nan (222)',
 'nan (56)',
 'nan (228)',
 'Dmitrii Bashkirov',
 'KS',
 'Andrey Pisankin',
 'Okafor Ikenna',
 'Dmtiry Grankin',
 'Дмитрий Гранкин',
 'MJ',
 

In [12]:
coworkers = [
    'Dmitriy Grankin',
    'Sergey Ryabenko',
    'Olga Nemirovskaya',
    'Alex Shevliakov',
    'Nick Frolov',
    'Jenish Mursidinov',
    'Dmitry Grankin',
]

In [13]:
context = f"""
Available topics: {', '.join(topics)}
Topic types: {', '.join(topic_types)}
Speakers: {', '.join(speakers)}
Content: {' '.join(samped_df.to_csv())}
"""

In [14]:
knowledge = """

our product name is Vexa
exclude coworkers if asking about users


"""

In [15]:
speaker_groups = ['coworkers']

In [26]:
messages = [
    system_msg(f'knwoledge: {knowledge},  speaker groups: {speaker_groups}, context {context}' ),
    user_msg('what users say about us? ')
            
            ]

In [27]:
from IPython.display import display,clear_output

In [28]:
async for plan in SearchQuery.call(messages):
    clear_output()
    print(plan, flush=True)


None


In [25]:
plan

In [20]:
# Initialize the sampler with your DataFrame
sampler = WeightedSampler(
    df=df,
    text_columns=['summary', 'details','topic_name'],  # Specify which columns to use for semantic search
    date_column='meeting_timestamp'
)

Building vector index...


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

In [21]:
coworkers

['Dmitriy Grankin',
 'Sergey Ryabenko',
 'Olga Nemirovskaya',
 'Alex Shevliakov',
 'Nick Frolov',
 'Jenish Mursidinov',
 'Dmitry Grankin']

In [22]:

# Now sampler is an instance, not a class
# Assuming 'plan' is your SearchQuery instance
sampled_df = sampler.sample(
    query=plan.query,  # List of query strings
    exclude_speakers=coworkers if 'coworkers' in plan.exclude_speakers  else plan.exclude_speakers,
    include_speakers=plan.include_speakers,
    topic_type=plan.topic_type,
    topic_name=plan.topic_name,
    n_samples=100,
    mode='combined',
    recency_weight=0.1,
    similarity_weight=0.1,
    filter_weight=1.
)

# Sort by combined score to see most relevant results first
sorted_results = sampled_df.sort_values('combined_score', ascending=False)

AttributeError: 'NoneType' object has no attribute 'query'

In [23]:
plan.model_dump()

AttributeError: 'NoneType' object has no attribute 'model_dump'

In [54]:
sorted_results

,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,other_speakers,filter_score,recency_score,similarity_score,combined_score
946,4,Plan to interview users who have stopped using...,Olga Nemirovskaya suggested reaching out to bo...,"Olga Nemirovskaya: на этой неделе, например, ...",Conduct user interviews for feedback,task,1be62b63-99ea-4aea-b284-a2ed45e44b82,2024-10-08 11:25:07.510000,Olga Nemirovskaya,[Dmitriy Grankin],2.0,0.000875,0.733700,0.000380
3139,7,Olga emphasized the need to clearly define the...,Identifying the right audience is crucial for ...,"Olga Nemirovskaya: Тогда, если мы сейчас хоти...",Target Audience Identification,task,3ead8b34-f5de-42ef-8d28-5dd64a2b8669,2024-09-16 09:47:34.620000,Olga Nemirovskaya,[Dmitriy Grankin],2.0,0.000097,0.603516,0.000378
1858,4,Reach out to users who registered but did not ...,The team will use email to contact users and i...,Olga Nemirovskaya: Хорошо. У нас же есть все ...,Contact users for onboarding feedback,task,733a72e0-cca9-4d47-b5e9-628aa1849556,2024-09-02 11:32:46.540000,Olga Nemirovskaya,[Dmitriy Grankin],2.0,0.000024,0.593231,0.000378
2993,4,The team discussed using SEO strategies to ext...,"By optimizing content for search engines, the ...","Olga Miller: И вот это вот то, что по минимал...",SEO Strategies for Use Cases,task,e2f3a86b-e94d-4967-8dad-89ac78a0617c,2024-09-13 10:17:35.355999,Dmitry Grankin,"[OM, DG, Olga Miller]",2.0,0.000072,0.585791,0.000377
1517,9,The team will keep track of new user registrat...,Daily monitoring of user registrations is esse...,Dmitriy Grankin: Типа там где-нибудь по одном...,Monitor user registrations and engagement,task,2ef8bcc3-d3fd-473a-a7c8-865873cb038c,2024-08-29 10:11:26.570000,Dmitriy Grankin,[Olga Nemirovskaya],2.0,0.000016,0.575901,0.000377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,11,FastAPI is a modern web framework for building...,It is used to create the API for the applicati...,,FastAPI,technology,c1f4044d-a8ce-4dc1-9383-2c066c37c351,2024-09-02 13:14:41.720000,Andrey Pisankin,[Dmitriy Grankin],1.0,0.000024,0.268219,0.000188
2468,17,"Docker is a platform for developing, shipping,...",He uses Docker as part of his development work...,"Юрий Б: По скиллам, соответственно, знаком с ...",Docker,product,472fca20-8cf3-4910-b08d-88b93d4919ef,2024-09-06 16:22:35.780000,ЮБ,"[Юрий Б, DG, Dmitriy Grankin]",1.0,0.000036,0.246868,0.000188
928,14,Training material refers to the content and re...,David mentioned the minimal budget and proof o...,David Sterry: It's view it more of as a proof...,training material,task,cdf9078d-87e0-49cb-9b62-71596961f436,2024-10-08 10:33:20.200000,David Sterry,"[Dmitry Grankin, Nadya Gorodetskaya]",1.0,0.000875,0.216466,0.000187
2787,36,Work-Life Balance refers to the equilibrium be...,The team discusses the importance of maintaini...,Dmitriy Grankin: В этом процессе и как раз та...,Work-Life Balance,concept,cde02f21-5001-4f19-b477-5dc4a04017fd,2024-09-11 10:57:53.150000,DG,"[Dmitriy Grankin, Sergey Ryabenko, Xut tuX, SR...",1.0,0.000059,0.216729,0.000187
